In [53]:
import cv2
import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
import holoviews as hv
import EzTrackFunctions as ez
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import FreezeAnalysis_Functions as fz

# Defining Functions & Variables

In [2]:
def find_file_with_strings(root_path, strings_list, dont_include_suffix):
    strings_list = [s.lower() for s in strings_list]
    for root, dirs, files in os.walk(root_path):
        for file in files:
            # ignore files that start with a dot (annoying lock files)
            if file.startswith("."):
                continue
            else:
                file_path = os.path.join(root, file).lower()
                if all(string.lower() in file_path for string in strings_list) and dont_include_suffix.lower() not in file_path:
                    return os.path.join(root, file)
    return None

Olena's Important Columns in "StartInfo" CSVs
Notes:
 - please name videos based on what mice they are, don't include the name of the two mice that were in both chambers b/c that gets cumbersome to memorize which is which based on what camera they came from (Cam1 vs. Cam2), all names of files/(their directories leading to it) should be standalone

1. type of experiment columns (conditioning, ..., renewal)
2. camera_type
3. animal_id (will only need this info for video name)
4. chamber
5. experimental_group (important for grouping averages)

Patrick's Important Columns in "StartInfo" CSVs

1. type of experiment columns (conditioning, ..., renewal)
2. animal_id
3. chamber
4. experimental_group

Ozge's Important Columns in "StartInfo" CSVs

1. type of experiment columns (conditioning, ..., renewal)
2. animal_id
3. chamber
4. experimental_group

In [7]:
# this will determine some parameters going into the automated analysis
person = "Olena"
# Adjust these parameters based on trial and error
FreezeThresh = 180 
MinDuration = 40
# this should be the suffix of your info sheet, if not, change it
info_sheet_suffix  = "_FC_startime_info.csv"
# this should be the suffix of your eztrack output, if not, change it
eztrack_output_processed_suffix = "FreezingOutput_processed.csv"
# your root path
PERSON_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group"
# your calibration root path, if the same as above, c&p the above
CALIBRATION_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/calibration"
# how much time after the event occurred do you want to get? in seconds
half_time_window = 2.5
# most likely you don't need to change this
calibrate_video_what_frame_to_start = 0
number_of_frames_to_calibrate = 600
h,w = 300,1000
dsmpl = 1
vid_d_start = 0
# change if you want to track another event, make sure name matchs column name
event_tracked = 'CS ON'

In [14]:
# defining columns we will be looking at for each person
experiment_cols = {
        "Olena": ["conditioning", "extinction_1", "extinction_2", "retrieval", "late_retrieval", "renewal"],
        "Ozge": ["conditioning", "extinction_1"],
        "Patrick": ["conditioning", "extinction_1", "retrieval"],
    }

general_cols = {
    "Olena":["animal_id", "camera_type"],
    "Ozge": ["animal_id"],
    "Patrick": ["animal_id"],
}

# we also focus on : chamber and experimental_group columns but we don't define that here

In [10]:
start_info_filepath = os.path.join(PERSON_ROOT, f"{person.lower()}{info_sheet_suffix}")
print(f"start_info_filepath: {start_info_filepath}")

df_start_info = pd.read_csv(start_info_filepath)
df_start_info.head()

start_info_filepath: /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/olena_FC_startime_info.csv


,animal_name,animal_id,chamber,experimental_group,camera_id,camera_type,conditioning,extinction_1,extinction_2,retrieval,late_retrieval,renewal
0,C10208,hSyn-AS-Gi-1,A,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
1,C10208,hSyn-AS-Gi-1,A,AS-Gi,1,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
2,C10209,hSyn-AS-Gi-2,C,AS-mCherry,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
3,C10208,hSyn-AS-Gi-2,A,AS-Gi,2,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
4,C10210,hSyn-AS-Gi-3,B,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000


In [21]:
def time_to_frames(time, fps):
    time_str = str(time)
    if ":" in time_str:
        # Convert time in minutes to time in seconds
        minutes, seconds = map(int, time_str.split(":"))
        time_in_seconds = 60 * minutes + seconds
    else:
        time_in_seconds = int(time)

    # Calculate number of frames based on time and fps
    frames = int(time_in_seconds * fps)

    return frames

# Calibration & FreezeAnalysis

In [ ]:
# going through experiment for this person
for experiment in experiment_cols[person]:
    print(experiment)
    # now go through each mouse for this person
    for index, mouse_row in df_start_info.iterrows():
        print(mouse_row["animal_id"])
        look_up_list = []
        look_up_list.append(experiment)
        
        for col in general_cols[person]:
            look_up_list.append(mouse_row[col])
        print(look_up_list)

        # find video given a list of strings it should contain
        vid_found = find_file_with_strings(PERSON_ROOT, look_up_list)
        print("vid_found: ", vid_found)
        # get experimental group
        experimental_group = mouse_row["experimental_group"]

        # Get fps of video
        vid_opencv_obj = cv2.VideoCapture(vid_found)
        fps = vid_opencv_obj.get(cv2.CAP_PROP_FPS)
        print("Frames per second:", fps)
        # if look_up_list contains the element 'usb', don't divide by 2,
        # otherwise, divide by 2, i think? yea bc eztrack downsamples by 2 for non-usbs
        if 'usb' in look_up_list:
            fps_eztrack_adjusted = fps
        else:
            fps_eztrack_adjusted = fps / 2 # bc eztrack downsamples by 2
        vid_opencv_obj.release()

        # get correction time in seconds to subtract from total frames
        correction_time_in_frames = math.ceil(time_to_frames(mouse_row[experiment], fps_eztrack_adjusted))
        print("correction_time_in_frames: ", correction_time_in_frames)

        # find calibration video
        chamber = mouse_row["chamber"]

        #to find calibration for extinction_get rid of _1 and _2
        if "_1" in experiment or "_2" in experiment:
            experiment_cali = experiment[:-2]
        else:
            experiment_cali = experiment

        calibration_vid_file_name = find_file_with_strings(CALIBRATION_ROOT, [chamber, experiment_cali])
        print("calibration_vid_file_name: ", calibration_vid_file_name)

        video_dict = {
            'dpath'   : CALIBRATION_ROOT,  
            'file'    : calibration_vid_file_name.split("/")[-1],
            'start'   : calibrate_video_what_frame_to_start, 
            'end'     : None,
            'dsmpl'   : dsmpl,
            'stretch' : dict(width=1, height=1),
            'cal_frms' : number_of_frames_to_calibrate
            }

        img_crp, video_dict = fz.LoadAndCrop(video_dict)
        ####### CALIBRATION #######
        cal_dif_avg, percentile, mt_cutoff = fz.calibrate_custom(video_dict, cal_pix=10000, SIGMA=1)

        ####### FREEZE ANALYSIS #######
        cap = cv2.VideoCapture(vid_found)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("length of vid: ", length)

        vid_d_end = length

        video_dict = {
            'dpath'   : PERSON_ROOT,  
            'file'    : vid_found.split("/")[-1],
            'fpath'   : vid_found,
            'start'   : vid_d_start, 
            'end'     : vid_d_end,
            'dsmpl'   : dsmpl,
            'stretch' : dict(width=1, height=1)
        }

        Motion, frames_processed = fz.Measure_Motion(video_dict, mt_cutoff, SIGMA=1)  
        plt_mt = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
            height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
        plt_mt

        Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
        fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)
        print('Average Freezing: {x}%'.format(x=np.average(Freezing)))

        vid_name_no_ext = vid_found.split("/")[-1].split(".")[0]
        freezing_result_filename = f"{vid_name_no_ext}_FreezingOutput.csv"
        freezing_result_path = vid_found.replace(vid_found.split("/")[-1], freezing_result_filename)

        # this processing includes frame correction from when experiment started
        processed_freezing_out_path = freezing_result_path.replace(".csv", "_processed.csv")

        df_freezing_out = pd.read_csv(freezing_result_path)
        #finding the timing file for this person and experiment
        timing_filepath = find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment])
        df_timing = ez.timing_file_processing(find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment]), fps_eztrack_adjusted, correction_time_in_frames)
        df_aligned = ez.freezing_alignment(df_freezing_out, df_timing)

        print("processed_freezing_out_path: ", processed_freezing_out_path)
        df_aligned.to_csv(processed_freezing_out_path, index=False)
        

# Plotting

In [41]:
def find_directory_with_strings(strings, root_directory):
    # Convert strings to lowercase
    strings_lower = [s.lower() for s in strings]

    # Iterate over directories in root directory
    for dirpath, _, _ in os.walk(root_directory):
        # Convert directory path to lowercase
        dirpath_lower = dirpath.lower()
        
        # Check if all strings are in the directory path
        if all(s in dirpath_lower for s in strings_lower):
            return dirpath

    # If no directory is found, return None
    return None

def get_adjusted_fps(video_file_path, camera_type):
    # Open video file
    cap = cv2.VideoCapture(video_file_path)

    # Get FPS
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Adjust FPS if camera type is not 'usb'
    if camera_type != 'usb':
        fps /= 2

    # Release video capture object
    cap.release()

    # Return adjusted FPS
    return fps

def extract_mouse_id_olena(filename):
    match = re.search(r"(hSyn-AS-Gi-\d+)", filename)
    if match:
        return match.group(1)
    else:
        return None


def extract_mouse_id_ozge(filename):
    match = re.search(r"C\d+", filename)
    if match:
        return match.group()
    else:
        return None

def extract_mouse_id_patrick(filename):
    return filename.split("_")[0]

def remove_substring(original_string, substring):
    return original_string.replace(substring, "")

In [56]:
only_pull_columns = ["animal_id", "experimental_group", "camera_type"]
groupby_cols = ["experimental_group", "camera_type"]
# going through experiment for this person
for experiment in experiment_cols[person]:
    print("experiment: ", experiment)
   
    opsin_group_colors = [mcolors.to_hex((random.random(), random.random(), random.random())), 
                            mcolors.to_hex((random.random(), random.random(), random.random()))]

    timing_filepath = find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment])
    df_timing = pd.read_csv(timing_filepath)
    cs_nums = range(1, len(df_timing) + 1)


    experimental_groups_df = df_start_info.loc[:, only_pull_columns]
    #print(experimental_groups_df.head())
    grouped_df = experimental_groups_df.groupby(groupby_cols).agg(lambda x: list(x))
    d_from_df = grouped_df.T.to_dict(orient='list')

    # flatten 2d array
    for key_1, values in d_from_df.items():
        d_from_df[key_1] = values[0]

    print("d_from_df: ")
    #print(json.dumps(d_from_df, indent=4))
    
    # need to see what d_from_df prints me out so i can determine data structure can be used to search for
    # a videos directory (experiment->camera_type) and then search for the video file name and see
    # in what opsin group it belongs in
    # is separated by tuples of opsin and camera type
    # got: experiment, camera_type (key), animal_id(value in list)

    # going through each grouping of mice per cam and experiment
    for key_2, values in d_from_df.items():
        print("CURRENT GROUP:", key_2)
        camera = key_2[1]
        if person == "Olena":
            curr_dir = find_directory_with_strings([experiment, camera], PERSON_ROOT)
        else:
            curr_dir = find_directory_with_strings([experiment], PERSON_ROOT)

        #d_from_df is grouped by experimental group and camera type
        # however, we search for vids under the same experiment and camera type for the person
        # which means we willbe able to separate by experimental group

        # new d will be created of avgs of groups
        d_groups = {}   
        num_mice = 0

        try:
            for file in os.listdir(curr_dir):
                if eztrack_output_processed_suffix in file:
                    #print("file: ", file)
                    # finding the name of animal embedded in the vid name
                    if person == "Olena":
                        mouse = extract_mouse_id_olena(file)
                    elif person == "Ozge":
                        mouse = extract_mouse_id_ozge(file)
                    elif person == "Patrick":
                        mouse = extract_mouse_id_patrick(file)
                    
                    #print(mouse)
                    experimental_group = None

                    # check if mouse is in one of the opsin groups
                    for key_3, values in d_from_df.items():
                        if mouse in d_from_df[key_3]:
                            experimental_group = key_3[0] 
                            # this will be true for olena & everyone else?
                            
                    #print(mouse, ":", experimental_group)
                    filepath = os.path.join(curr_dir, file)
                    filepath_video = remove_substring(filepath, "_" + eztrack_output_processed_suffix) + ".avi"
                    #print("filepath_video: ", filepath_video)
                    fps_adjusted = get_adjusted_fps(filepath_video, camera)
                    #print("fps_adjusted: ", fps_adjusted)
                    
                    df = pd.read_csv(filepath)
                    frame_lst = list(df["Frame"])

                    timestamps_lst = list(df["Timestamps"])
                    
                    # stamped_lst is the x
                    stamped_lst = ez.overlap_two_lists(frame_lst, timestamps_lst)

                    # modify y to just be binary and not 0 and 100
                    freezing_lst = ez.lst_to_binary_lst(list(df["Freezing"]))
                    #print(freezing_lst)

                    # half_time_window is in seconds
                    x, proportions = ez.bin_data(frame_lst, timestamps_lst,freezing_lst, half_time_window = half_time_window, fps=fps_adjusted, event_tracked=event_tracked)
                    #list_of_freezing_props_all_mice.append(proportions)

                    # add to d
                    if experimental_group in d_groups:
                        d_groups[experimental_group].append(proportions)
                    else:
                        d_groups[experimental_group] = []
                        d_groups[experimental_group].append(proportions)

                    num_mice += 1
                #break
            print("d_groups: ")
            print(json.dumps(d_groups, indent=4))

            count = 0
            fig, ax = plt.subplots()
            for key_4 in d_groups:
                print("key_4: ", key_4)

                # Convert the list of lists to a NumPy array
                array_of_lists = np.array(d_groups[key_4])
                print(array_of_lists)

                # Calculate the average of the array along the columns (axis=0)
                average = np.mean(array_of_lists, axis=0)
                #print(average)

                # Calculate the standard deviation of the array along the columns (axis=0)
                std_deviation = np.std(array_of_lists, axis=0)
                std_error = [std / math.sqrt(num_mice) for std in std_deviation]

                ax.plot(cs_nums, average, label=key_4, color=opsin_group_colors[count])
                plt.errorbar(cs_nums, average, yerr = std_error, fmt='-o', color=opsin_group_colors[count], capsize=3)

                count += 1

            outfilename = f"{experiment}_halftimewdw{half_time_window}_plot.png"
            outpath = "/".join(filepath.split("/")[:-1]) + "/" + outfilename
            print(outpath)
            
            ax.set_title(f"Proportion of Freezing - {experiment} (n={num_mice})")
            ax.set_ylabel(f"Proportion")
            ax.set_xlabel(f"CS #")
            plt.legend()
            fig.savefig(outpath)
            plt.close()
        except Exception as e:
            print(e)
            continue
        #break

experiment:  conditioning
d_from_df: 


TypeError: keys must be str, int, float, bool or None, not tuple